In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI

# 我们仍旧使用 GPT-4o，这是一个支持多模态的 LLM
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,  # 让我们的 LLM 最准确
    max_tokens=2048,
    timeout=None,
    max_retries=2,
)

我们之前学习过了语义搜索和关键词搜索，这两者都是基于原始数据（非结构化文本）。因为本身就没有一种良好的搜索算法去进行相似度计算，因此 LLM 在这件事情上是工作良好的。但是如果我们面对的是数据库之类的搜索，由于本身数据就是带有结构化信息，成熟的 SQL 可以做到100%精确的查找，此时我们 LLM 反倒不能因为 100% 的确定性而不适用。

![information retrieval](https://python.langchain.com/assets/images/retrieval_concept-2bcff1b2518f194b34eaf472ac748ffa.png)

因此我们思考一下，能否让 LLM 为我们生成 SQL 代码，这样 LLM 可以理解我们的查找要求，并且生成一份显性的查询语句提供我们搜索，这样能做到完全的透明，并且很容易在 LLM 生成的 SQL 上面进行调整，省去从头构建 SQL 的麻烦。

尽管用户通常更喜欢使用自然语言与检索系统交互（比如跟 LLM 说一声想要什么数据，LLM 就能理解并帮你取来），但这些系统可能需要特定的查询语法或受益于某些关键字。[查询分析](https://python.langchain.com/docs/concepts/retrieval/#query-analysis)充当原始用户输入和优化搜索查询之间的桥梁。查询分析的一些常见应用包括：

* 查询重写：可以重写或扩展查询以改进语义或词汇搜索。以下是在非结构化数据检索中使用模型进行查询分析的一些主要好处：
    * 查询澄清：模型可以重新表述模棱两可或措辞不当的查询，以便更加清晰；
    * 语义理解：它们可以捕捉查询背后的意图，超越文字关键字匹配；
    * 查询扩展：模型可以生成相关的术语或概念以扩大搜索范围；
    * 复杂查询处理：他们可以将多部分问题分解为更简单的子查询。
* 查询构建：搜索索引可能需要结构化查询（例如数据库的 SQL）。


In [2]:
# 以下是查询重写中的查询分解步骤
from typing import List

from pydantic import BaseModel, Field
from langchain_core.messages import SystemMessage, HumanMessage

# Define a pydantic model to enforce the output structure
class Questions(BaseModel):
    questions: List[str] = Field(
        description="A list of sub-questions related to the input query."
    )

# Create an instance of the model and enforce the output structure
structured_model = llm.with_structured_output(Questions)

# Define the system prompt
system = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n"""

# Pass the question to the model
question = """What are the main components of an LLM-powered autonomous agent system?"""
questions = structured_model.invoke(
    [SystemMessage(content=system)]+[HumanMessage(content=question)]
)

In [15]:
for index, question in enumerate(questions.questions):
    print(f"Step {index + 1}: {question}")

Step 1: What is an LLM and how does it function within an autonomous agent system?
Step 2: What are the key components of an autonomous agent system?
Step 3: How does an LLM interact with other components in an autonomous agent system?
Step 4: What role does data play in the functioning of an LLM-powered autonomous agent system?
Step 5: How is decision-making handled in an LLM-powered autonomous agent system?
Step 6: What are the input and output mechanisms for an LLM in an autonomous agent system?
Step 7: How is the performance of an LLM-powered autonomous agent system evaluated?
Step 8: What are the challenges in integrating LLMs into autonomous agent systems?
Step 9: What are some examples of LLM-powered autonomous agent systems in use today?


可以发现，对于一个这样的问题“What are the main components of an LLM-powered autonomous agent system?”，LLM 可以自己向自己提问，逐步分解这个问题直到把所有的要点分析出来。超越关键字进行联想和搜索，并尝试给于更加合理且详尽的解释。

第二点也就是我们的[查询语句构建](https://python.langchain.com/docs/concepts/retrieval/#query-construction)，这一部分已经有很多团队在做，比如[将自然语言转换为关系数据库的 SQL](https://paperswithcode.com/task/text-to-sql)、[将自然语言转换为图形数据库的密码](https://neo4j.com/labs/neodash/2.4/user-guide/extensions/natural-language-queries/)，可以去参考网站上面的模型内容。

> 例如 Text-to-SQL: Text-to-SQL is a task in natural language processing (NLP) where the goal is to automatically generate SQL queries from natural language text. The task involves converting the text input into a structured representation and then using this representation to generate a semantically correct SQL query that can be executed on a database.

这其中也不乏一些技巧，比如 Self-query，一种技术将 LLM 将用户输入转换为两部分：(1) 一个用于语义查找的字符串，(2) 一个用于与之配合的元数据过滤器。这很有用，因为问题通常与文档的元数据有关（而不是内容本身）。示例代码如下：

```python
metadata_field_info = schema_for_metadata
document_content_description = "Brief summary of an exam."
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)
```